In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

import pickle

from util import T, mean_err, M_pi, M_K
from main_diagrams import typeI_typeII_t_seps, calc_amp_multiple_trajs, calc_amp_sum_tv

# trajs = range(2100, 2150, 10)
trajs = sorted([int(f[:4]) for f in os.listdir('./Kaon_results/')])

# trajs = [t for t in trajs if t<1290]  # 1010 - 1270 amplitude is 20% larger than 1900 - 2150

print('trajs:', trajs)
print(f'Number of trajectories: {len(trajs)}')

trajs: [1010, 1030, 1040, 1050, 1070, 1080, 1090, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820, 1840, 1850, 1860, 1870, 1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100, 2110, 2120, 2130, 2140, 2150, 2160, 2170, 2180, 2190, 2200, 2210, 2220, 2230, 2240, 2250]
Number of trajectories: 117


### Amplitude v.s. t_v  for every tK 

In [18]:
amp_Hw = calc_amp_multiple_trajs(trajs, avg_tK=False)
amp_Hw_avg, amp_Hw_err = mean_err(amp_Hw)

# tv_min, tv_max = -4, 8
tv_min, tv_max = -4, 6

fig = go.Figure()
for i, t_sep in enumerate(typeI_typeII_t_seps):
    y = np.concatenate([amp_Hw_avg[i][tv_min:], amp_Hw_avg[i][:tv_max+1]])
    err_y = np.concatenate([amp_Hw_err[i][tv_min:], amp_Hw_err[i][:tv_max+1]])
    fig.add_trace(go.Scatter(x=list(range(tv_min, tv_max+1)), y=y, 
                             error_y=dict(type='data', array=err_y),
                             name=f'tsep={t_sep}')
                 )
fig.add_shape(dict(type='line', line_dash='dot',
      y0= -0.02047, y1=-0.02047,
      x0= tv_min, x1=tv_max))
fig.update_layout(title_text='Amplitude of Q1+Q2', width=600, height=400)
fig.update_xaxes(title_text='t_v')
fig.update_yaxes(title_text='Amplitude')
fig.show()

### Amplitude v.s. t_v  after averaging over tK 

In [19]:
amp_Hw = calc_amp_multiple_trajs(trajs, avg_tK=True)
amp_Hw_avg, amp_Hw_err = mean_err(amp_Hw)

# tv_min, tv_max = -5, 8
tv_min, tv_max = -5, 6

fig = go.Figure()
y = np.concatenate([amp_Hw_avg[tv_min:], amp_Hw_avg[:tv_max+1]])
err_y = np.concatenate([amp_Hw_err[tv_min:], amp_Hw_err[:tv_max+1]])
fig.add_trace(go.Scatter(x=list(range(tv_min, tv_max+1)), y=y, 
                         error_y=dict(type='data', array=err_y))
)

fig.add_shape(dict(type='line', line_dash='dot',
      y0= -0.02047, y1=-0.02047,
      x0= tv_min, x1=tv_max))

fig.update_layout(title_text='', width=600, height=400)
fig.update_layout(title='Amplitude before subtracting pion contribution')
fig.update_xaxes(title_text='t_v')
fig.update_yaxes(title_text='Amplitude')
fig.write_image('amp_vs_tv_before_pion.pdf')
fig.show()

### Directly fit |pi^0> contribution

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [29]:
# Fit pion contribution

from scipy.optimize import curve_fit

t_min, t_max = 4, 14

def func(x, C, M, b):
    return C * np.exp(x * M) + b

# def func(x, C, b):
#     return C * np.exp(x * (M_K - M_pi)) + b 

# y = amp_Hw_avg[:10]  
# x = np.arange(10)
# y_err = amp_Hw_err[:10]
y = amp_Hw_avg[t_min:t_max+1]  
x = np.arange(t_min, t_max+1)
y_err = amp_Hw_err[t_min:t_max+1]
popt, pcov = curve_fit(func, x, y)
# popt, pcov = curve_fit(func, x, y, sigma=y_err) # fitting is worse with y_err

print(popt)  
mass = popt[1]
mass_err = np.sqrt(pcov[1, 1])
print(f'Fitted mass: {mass:.4f} GeV with error {mass_err:.4f} GeV; should be M_K - M_pi = 0.3639 GeV')
print(f'Coefficient {popt[0]:.4f} with error {np.sqrt(pcov[0, 0]):.4f}')
print(f'Bias {popt[2]:.4f} with error {np.sqrt(pcov[2, 2]):.4f}')


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, error_y=dict(type='data', array=y_err), mode='markers',
                        name='Amplitude'))

x = np.linspace(0, max(x), 100)
y = func(x, *popt)
fig.add_trace(go.Scatter(x=x, y=y, name='Fitted curve'))
fig.update_layout(title='Fitting pion contribution with exponential function')
fig.update_xaxes(title_text='t_v')
fig.update_yaxes(title_text='Amplitude')
fig.write_image('amp_fitting.pdf')
fig.show()

[-0.00627795  0.36275875  0.00292698]
Fitted mass: 0.3628 GeV with error 0.0152 GeV; should be M_K - M_pi = 0.3639 GeV
Coefficient -0.0063 with error 0.0014
Bias 0.0029 with error 0.0120


### Sum over t_v

In [4]:
amp_Hw = calc_amp_sum_tv(trajs, tv_min = -4, tv_max=3)
print(f'amp_Hw summed over t_v and averaged over all trajs: {amp_Hw:.4f}')

amp_Hw summed over t_v and averaged over all trajs: -0.0672


# Subtract pion

Without including disconnected diagrams (i.e. Type V diagrams), there are two pions, one has isospin 0, and the other one has isospin 1.  

Equivalently, we can take the new view that there are two particles "uBar u" and "dBar d", and subtract <JJ | uBar u> <uBar u| Hw|K> + <JJ | dBar d> <dBar d| Hw|K>

i.e. we change <JJ | pi^0> <pi^0| Hw|K>  ->  <JJ | uBar u> <uBar u| Hw|K> + <JJ | dBar d> <dBar d| Hw|K>

In [5]:
# Subtract <JJ | pi^0> <pi^0| Hw|K>   # should do this after including disconnected diagrams

# Euv_JJ_pi = 0.0937  # From pion_state.ipynb
# Euv_JJ_pi_err = 0.0034
# pi_Hw_KL = -0.0076 
# pi_Hw_KL_err = 2.27e-4 

# t_v = np.arange(T)
# exp_factor = np.exp((M_K - M_pi) * t_v)
# exp_factor = np.where(t_v>=20, 0, exp_factor)

# coef = 1 / (2 * M_pi) * Euv_JJ_pi * pi_Hw_KL
# coef = - coef    # Pion interpolator has coefficient "i"

# print(coef)   # 2 times smaller than direct fitting; sign is also wrong
# pion_contrib = coef * exp_factor  

# amp_after_pion = amp_Hw_avg - pion_contrib

In [6]:
# Subtract <JJ | pi^0> <pi^0| Hw|K> (0.0025)  + <JJ | eta_l> <eta_l| Hw|K> (-0.0082) ~= -0.0059
# Subtract <JJ | uBar g5 u> <uBar g5 u| Hw|K> + <JJ | dBar g5 d> <dBar g5 d| Hw|KL>

def calc_uBar_u_dBar_d_contrib(trajs):

    from JJ_uBar_u import calc_JJ_uBar_u_avg_tpi_sum_tu
    from qBar_q_Hw_K import calc_qBar_q_Hw_K_avg_tsep_tx

    Euv_JJ_uBar_u = calc_JJ_uBar_u_avg_tpi_sum_tu(trajs, MAX_ut=10)
    Euv_JJ_dBar_d = Euv_JJ_uBar_u / 4.

    uBar_u_Hw_KL, dBar_d_Hw_KL = calc_qBar_q_Hw_K_avg_tsep_tx(trajs, min_tsep=18, min_tx=8, max_tx=15)

    coef = 1 / (2 * M_pi) * (Euv_JJ_uBar_u * uBar_u_Hw_KL + Euv_JJ_dBar_d * dBar_d_Hw_KL)
    coef = - coef    # !!interpolators must have coefficient "i" to be hermitian:  (i uBar g5 u) and  (i dBar g5 d)

    t_v = np.arange(T)
    exp_factor = np.exp((M_K - M_pi) * t_v)
    exp_factor = np.where(t_v>=20, 0, exp_factor)
    pion_contrib = coef * exp_factor    # shape (64,)
    return coef, pion_contrib

In [7]:
coef, pion_contrib = calc_uBar_u_dBar_d_contrib(trajs)
print(f'coef: {coef}')   # should be close to direct fitting; sign is correct
amp_after_pion = amp_Hw_avg - pion_contrib

coef: -0.005790706975651993


In [8]:
tv_min, tv_max = -4, 8

fig = go.Figure()

y = np.concatenate([amp_after_pion[tv_min:], amp_after_pion[:tv_max+1]])
err_y = np.concatenate([amp_Hw_err[tv_min:], amp_Hw_err[:tv_max+1]])  # FIXME: only considered error of main diagrams
fig.add_trace(go.Scatter(x=list(range(tv_min, tv_max+1)), y=y, 
                         error_y=dict(type='data', array=err_y))
)

fig.add_shape(dict(type='line', line_dash='dot',
      y0= -0.02047, y1=-0.02047,
      x0= tv_min, x1=tv_max))
# fig.update_layout(title_text='Amplitude after subtracting pion', width=600, height=400)
fig.update_layout(title_text='Amplitude after subtracting pion contribution', width=600, height=400)
fig.update_xaxes(title_text='t_v')
fig.update_yaxes(title_text='Amplitude')
fig.write_image('amp_vs_tv_after_pion.pdf')
fig.show()

### Total amplitude after subtracing uBar u and dBar d

In [9]:
def total_amp_after_subtract_uBar_u_dBar_d(trajs, tv_min = -4, tv_max=3):
    amp_Hw = calc_amp_sum_tv(trajs, tv_min = -4, tv_max=3)
    _, pion_contrib = calc_uBar_u_dBar_d_contrib(trajs)
    pion_contrib = pion_contrib[:tv_max+1].sum()   # sum over tv_max
    
    rst = amp_Hw - pion_contrib
    return rst

In [10]:
amp = total_amp_after_subtract_uBar_u_dBar_d(trajs, tv_min = -4, tv_max=3)
print(f'total amplitude after subtracing uBar u and dBar d: {amp:.4f}. Should be -0.0205')

total amplitude after subtracing uBar u and dBar d: -0.0238. Should be -0.0205


# Jackknife

In [11]:
amps = []
N_trajs = len(trajs)
for idx in range(N_trajs):
    jk_trajs = trajs[:idx]  + trajs[idx+1:]  # remove one traj
    amp = total_amp_after_subtract_uBar_u_dBar_d(jk_trajs, tv_min = -4, tv_max=3)
    amps.append(amp)
    
amps = np.array(amps)
jk_mean = amps.mean()
jk_err = np.sqrt(amps.var(ddof=0) * (N_trajs-1))
print(f'Jackknife mean {jk_mean:.4f}, error {jk_err:.4f}. Should be -0.0205')

Jackknife mean -0.0238, error 0.0020. Should be -0.0205


# Eta intermediate state (Not ready to run yet)

In [ ]:
assert False

In [ ]:
def calc_amp_sBar_d(traj):
    data = pickle.load(open(f'./Kaon_results/{traj}.pkl', 'rb'))
    
    amp_sBar_d = sum([data[d] * coefficients[d] * hadron_coef for d in sBar_d_diagrams])
    amp_sBar_d = amp_sBar_d.real 
    
    return amp_sBar_d

amp_sBar_d = []
for traj in trajs:
    amp_sBar_d_one_traj = calc_amp_sBar_d(traj)   # shape (tsep, tx)
    amp_sBar_d.append(amp_sBar_d_one_traj)
amp_sBar_d = np.array(amp_sBar_d)  # (trajs, tsep, tx)


amp_sBar_d_avg, amp_sBar_d_err = mean_err(amp_sBar_d)

In [ ]:
c_s_Q1 = -0.002 # From c_s_eta.ipynb
c_s_Q2 = 0.003 # From c_s_eta.ipynb

# 1/np.sqrt(2) is the factor in K_L
amp_sBar_d_avg = amp_sBar_d_avg * (1. / np.sqrt(2)) * (C1 * c_s_Q1 + C2 * c_s_Q2)
amp_sBar_d_err = amp_sBar_d_err * (1. / np.sqrt(2)) * (C1 * c_s_Q1 + C2 * c_s_Q2)

amp_final = amp_after_pion + amp_sBar_d_avg

In [ ]:
plot_amplitude(amp_final, amp_Hw_err, title='Amplitude after subtracting pion and eta') # FIXME: add error of other terms